In [1]:
import axelrod as axl
from fractions import Fraction

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import itertools
import numpy as np

In [68]:
import opt_mo
import pandas as pd
import sympy as sym

Though Stewart and Plotkin tournament had a total of 19 strategies today we will consider only the memory
one ones. These are:
    
- cooperator
- defector
- random
- tit for tat
- grudger
- generous tit for tat
- win stay lose shift
- ZDGTFT2
- ZDExtort2
- joss

In [5]:
transitions = [(1, 1, 1, 1), (0, 0, 0, 0), (1/2, 1/2, 1/2, 1/2), (1, 0, 1, 0), (1, 0, 0, 0),
               (1, 1/3, 1, 1/3), (1, 0, 0, 1), (1, 1/8, 1, 1/4), (8/9, 1/2, 1/3, 0), (0.9, 0, 0.9, 0)]

In [6]:
len(transitions)

10

In [7]:
strategies = [axl.MemoryOnePlayer(p) for p in transitions]

In [8]:
ps = list(itertools.product(np.linspace(0, 1, 4), repeat=4))

In [9]:
players = [axl.MemoryOnePlayer(p) for p in ps]

In [10]:
number_of_players = len(strategies) + 1

In [11]:
edges = [(i, number_of_players - 1) for i in range(number_of_players - 1)]

Simulated values
----------------

In [12]:
simulated_scores = []
for player in players:
    tournament = axl.Tournament(players=strategies + [player], turns=200, 
                                repetitions=100, edges=edges)
    results = tournament.play(progress_bar=False)
    simulated_scores.append(np.mean(results.normalised_scores[-1]))

**Theoretical values**

In [13]:
theoretical_values = []
for player in ps:
    utility = -opt_mo.tournament_utility(player, transitions)
    theoretical_values.append(utility)

In [91]:
difference = [(simulated_scores[i] - theoretical_values[i], i) for i in range(len(simulated_scores))]
difference.sort()

In [92]:
display = difference[:10]

In [93]:
columns = [[*ps[i], theoretical_values[i], simulated_scores[i]] for _, i in difference[:10]]

In [94]:
labels = ['$p_1$', '$p_2$', '$p_3$', '$p_4$', r'$\sum\limits_{i = 1} ^ N u_q ^ {(i)} u(p)$', 
          r'$\sum\limits_{i = 1} ^ N u_q ^ {(i)} U(p)$']

In [95]:
table = pd.DataFrame(columns)
table.columns = labels

In [96]:
table.sort_values('$p_2$'). sort_values('$p_3$')

,$p_1$,$p_2$,$p_3$,$p_4$,$\sum\limits_{i = 1} ^ N u_q ^ {(i)} u(p)$,$\sum\limits_{i = 1} ^ N u_q ^ {(i)} U(p)$
4,0.0,0.000000,0.000000,0.666667,2.191268,2.200970
8,0.0,0.333333,0.000000,0.666667,2.199277,2.211010
1,0.0,0.000000,0.333333,1.000000,2.165901,2.173365
0,0.0,0.333333,0.333333,1.000000,2.158921,2.166100
6,0.0,0.000000,0.666667,1.000000,2.156748,2.166955
7,0.0,0.000000,0.666667,0.666667,2.145524,2.156880
3,0.0,0.333333,0.666667,1.000000,2.139927,2.149585
9,0.0,0.000000,1.000000,0.333333,2.186932,2.198790
2,0.0,0.333333,1.000000,1.000000,2.149520,2.157840
5,0.0,0.333333,1.000000,0.666667,2.157784,2.167540


In [97]:
table = table.replace(0.333333, sym.S(1) / 3)
table = table.replace(0.666667, sym.S(2) / 3)

In [98]:
tex = table.to_latex().replace('0.333333', r'$\frac{1}{3}$').replace('0.666667', r'$\frac{2}{3}$')

In [99]:
textfile = open('../tex/stewart_results.txt', 'w') 
textfile.write(tex)
textfile.close()